In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
train = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')
train.shape, test.shape

In [ ]:
train = train.dropna(axis='columns')
train.columns, len(train.columns)

In [ ]:
train.isnull().sum()

### choose columns : refrig, tamhog, hhsize, public, idhogar, dependency, hogar_total, overcrowding, bedrooms, agesq, Target

In [ ]:
train_feature = train[['refrig', 'tamhog', 'hhsize', 'public', 'idhogar', 'dependency', 'hogar_total', 'overcrowding', 'bedrooms', 'agesq']]
train_feature.head(3)

In [ ]:
train_feature.info()

* 연속형 : refrig, public, dependency, overcrowding, bedrooms
* 분류형 : tamhog, hhsize, hogar_total 

In [ ]:
train_feature['agesq'].value_counts()

In [ ]:
train_feature  = train_feature[['refrig', 'public', 'dependency', 'overcrowding', 'bedrooms', 'tamhog', 'hhsize', 'hogar_total']]
train_feature.head(3)

In [ ]:
train_feature['dependency'].value_counts()

In [ ]:
train_feature['dependency'] = train_feature['dependency'].replace(to_replace='yes', value=2)

In [ ]:
train_feature['dependency'] = train_feature['dependency'].replace(to_replace='no', value=6)

In [ ]:
train_feature.corr()

In [ ]:
train_heatmap = pd.concat([train['Target'],train_feature], axis='columns')
train_heatmap.corr()

In [ ]:
import seaborn as sns
sns.heatmap(train_heatmap.corr(),annot=True)

In [ ]:
train_onehot = pd.get_dummies(train_feature, columns=['tamhog', 'hhsize', 'hogar_total'])
train_onehot.shape

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(train_onehot)
X = scaler.transform(train_onehot)
X.shape

In [ ]:
Y = train['Target']
Y.shape

### Apply GridSearch

In [ ]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()

# class sklearn.model_selection.GridSearchCV(estimator, param_grid ...)
from sklearn.model_selection import GridSearchCV
params = {'max_depth':[10,15,20],
          'min_child_samples':[20,40,60],
          'subsample':[0.8,1],
         }
# map(func, param01, param02)

grid = GridSearchCV(lgbm, param_grid=params)
grid.fit(X,Y)

In [ ]:
grid.best_score_, grid.best_params_

In [ ]:
lgbm_grid = LGBMClassifier(max_depth=15, min_child_samples=40, subsample=0.8)
lgbm_grid.fit(X,Y)
lgbm_grid.score(X,Y)

### Apply KFold
class sklearn.model_selection.RepeatedStratifiedKFold(*, n_splits=5, n_repeats=10, random_state=None)
sklearn.model_selection.cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
kf = RepeatedStratifiedKFold(n_splits=10, n_repeats=5)

from sklearn.model_selection import cross_val_score
n_scores = cross_val_score(lgbm_grid, X, Y, scoring='accuracy', cv=kf)

In [ ]:
n_scores, n_scores.shape

In [ ]:
import numpy as np
np.mean(n_scores), np.std(n_scores)

In [ ]:
lgbm_grid.feature_importances_

In [ ]:
from lightgbm import plot_importance
print(train_onehot.columns)
plot_importance(lgbm_grid)

In [ ]:
Y_pred = lgbm_grid.predict(X)
from sklearn import metrics
print(metrics.classification_report(Y,Y_pred))